# Setup and run AWS instances

In [1]:
import boto3
import pandas as pd
import sys

In [18]:
# see what is the spot prices in regions
ec2 = boto3.client('ec2')
response = ec2.describe_regions()

{u'Regions': [{u'Endpoint': 'ec2.ap-south-1.amazonaws.com', u'RegionName': 'ap-south-1'}, {u'Endpoint': 'ec2.eu-west-2.amazonaws.com', u'RegionName': 'eu-west-2'}, {u'Endpoint': 'ec2.eu-west-1.amazonaws.com', u'RegionName': 'eu-west-1'}, {u'Endpoint': 'ec2.ap-northeast-2.amazonaws.com', u'RegionName': 'ap-northeast-2'}, {u'Endpoint': 'ec2.ap-northeast-1.amazonaws.com', u'RegionName': 'ap-northeast-1'}, {u'Endpoint': 'ec2.sa-east-1.amazonaws.com', u'RegionName': 'sa-east-1'}, {u'Endpoint': 'ec2.ca-central-1.amazonaws.com', u'RegionName': 'ca-central-1'}, {u'Endpoint': 'ec2.ap-southeast-1.amazonaws.com', u'RegionName': 'ap-southeast-1'}, {u'Endpoint': 'ec2.ap-southeast-2.amazonaws.com', u'RegionName': 'ap-southeast-2'}, {u'Endpoint': 'ec2.eu-central-1.amazonaws.com', u'RegionName': 'eu-central-1'}, {u'Endpoint': 'ec2.us-east-1.amazonaws.com', u'RegionName': 'us-east-1'}, {u'Endpoint': 'ec2.us-east-2.amazonaws.com', u'RegionName': 'us-east-2'}, {u'Endpoint': 'ec2.us-west-1.amazonaws.com',

In [34]:
# create keys
# need to set these from the above information
# works but a time off set between file
# us-east-2c

rg_name = 'us-east-2'
av_zone = 'us-east-2c'

subnet = 'subnet-0b8ec121'

kname = '6TestKey'
ec2 = boto3.resource('ec2', region_name=rg_name)
pth= '/Users/pgreisen/AWS/tmp_keys/'
projectname_key = pth+kname+'.pem'


try:
    key_pair = ec2.create_key_pair(KeyName=kname)
except:
    print "Key already exists",key_pair
    
KeyPairOut = str(key_pair.key_material)

# dump to file
with open(projectname_key,'w') as f:
    f.write(KeyPairOut)

Key already exists ec2.KeyPair(name='6TestKey')


In [35]:
ImageId='ami-c7ee5ca8'

In [36]:
filters = [{'Name':'tag:Name', 'Values':['VPN*']}]
vpcs = list(ec2.vpcs.filter(Filters=filters))
print vpcs

[]


In [37]:
import boto.vpc
import time
REGION_NAME = rg_name
AMI_ID = ImageId  # Amazon Linux AMI  
conn = boto.vpc.connect_to_region(REGION_NAME)

In [33]:
# List VPCs
filters = [{'Name':'tag:Name', 'Values':['VPN*']}]
vpcs = list(ec2.vpcs.filter(Filters=filters))

print ec2.vpcs.all
print ec2.vpcs.limit
print ec2.vpcs.filter

print conn.describe_vpc_attribute

for vpc in vpcs:
    response = conn.describe_vpcs(
        VpcIds=[
            vpc.id,
        ]
    )
    print(json.dumps(response, sort_keys=True, indent=4))

<bound method ec2.vpcsCollectionManager.all of ec2.vpcsCollectionManager(ec2.ServiceResource(), ec2.Vpc)>
<bound method ec2.vpcsCollectionManager.limit of ec2.vpcsCollectionManager(ec2.ServiceResource(), ec2.Vpc)>
<bound method ec2.vpcsCollectionManager.filter of ec2.vpcsCollectionManager(ec2.ServiceResource(), ec2.Vpc)>
<bound method VPCConnection.describe_vpc_attribute of VPCConnection:ec2.us-east-2.amazonaws.com>


In [38]:
import boto.vpc
import time

REGION_NAME = rg_name
AMI_ID = ImageId  # Amazon Linux AMI  
  
conn = boto.vpc.connect_to_region(REGION_NAME)


# Create a VPC
vpc = conn.create_vpc('10.0.0.0/16')
  
# Configure the VPC to support DNS resolution and hostname assignment
conn.modify_vpc_attribute(vpc.id, enable_dns_support=True)
conn.modify_vpc_attribute(vpc.id, enable_dns_hostnames=True)
  
# Create an Internet Gateway
gateway = conn.create_internet_gateway()
  
# Attach the Internet Gateway to our VPC
conn.attach_internet_gateway(gateway.id, vpc.id)
  
# Create a Route Table
route_table = conn.create_route_table(vpc.id)
  
# Create a size /16 subnet
subnet = conn.create_subnet(vpc.id, '10.0.0.0/24')
  
# Associate Route Table with our subnet
conn.associate_route_table(route_table.id, subnet) # subnet.id)
  
# Create a Route from our Internet Gateway to the internet
route = conn.create_route(route_table.id, '0.0.0.0/0', gateway.id)
  
# Create a new VPC security group
sg = conn.create_security_group('test',
                                'A test',
                                vpc.id)

# Authorize access to port 22 from anywhere
sg.authorize(ip_protocol='tcp', from_port=22, to_port=22, cidr_ip='0.0.0.0/0')

EC2ResponseError: EC2ResponseError: 400 Bad Request
<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>InvalidSubnetID.NotFound</Code><Message>The subnet ID 'Subnet:subnet-1b2dce73' does not exist</Message></Error></Errors><RequestID>627f4e31-f98d-4286-b3d7-725e1819d255</RequestID></Response>

In [28]:
print gateway.id

igw-4e89c127


In [29]:
# Ubuntu Server 16.04 LTS (HVM), SSD Volume Type - ami-cd0f5cb6
# Amazon Linux AMI 2017.09.0 (HVM), SSD Volume Type - ami-8c1be5f6
# Deep Learning AMI Ubuntu Version 2.4_Oct2017 - ami-37bb714d
# Deep Learning AMI Amazon Linux Version 3.3_Oct2017 - ami-78994d02
import boto3
# ec2 = boto3.resource('ec2')
ec2 = boto3.resource('ec2', region_name=rg_name)

instance = ec2.create_instances(
    ImageId="ami-c5062ba0", #ImageId,
    MinCount=1,
    MaxCount=1,
    KeyName=kname,
    Placement={'AvailabilityZone': av_zone },
    SecurityGroupIds=[sg.id],
    SubnetId=subnet.id,
    InstanceType='t2.nano')
print instance[0].id

ClientError: An error occurred (InvalidParameterValue) when calling the RunInstances operation: Value (us-east-2c) for parameter availabilityZone is invalid. Subnet 'subnet-20658548' is in the availability zone us-east-2a

In [19]:
import boto3
client = boto3.client('ec2')
addresses_dict = client.describe_addresses()
for eip_dict in addresses_dict['Addresses']:
    print eip_dict['PublicIp']

52.54.56.231


In [18]:
from collections import defaultdict

import boto3

"""
A tool for retrieving basic information from the running EC2 instances.
"""

# Connect to EC2
ec2 = boto3.resource('ec2')

# Get information for all running instances
running_instances = ec2.instances.filter(Filters=[{
    'Name': 'instance-state-name',
    'Values': ['running']}])


ec2info = defaultdict()
for instance in running_instances:
    for tag in instance.tags:
        if 'Name'in tag['Key']:
            name = tag['Value']
    # Add instance info to a dictionary         
    ec2info[instance.id] = {
        'Name': name,
        'Type': instance.instance_type,
        'State': instance.state['Name'],
        'Private IP': instance.private_ip_address,
        'Public IP': instance.public_ip_address,
        'Launch Time': instance.launch_time
        }
attributes = ['Name', 'Type', 'State', 'Private IP', 'Public IP', 'Launch Time']

for instance_id, instance in ec2info.items():
    for key in attributes:
        print("{0}: {1}".format(key, instance[key]))
    print("------")

In [30]:
ec2 = boto3.resource("ec2")
for vpc in ec2.vpcs.all():
    for subnet in vpc.subnets.all():
        print(vpc, "all:", subnet)
    for az in ec2.meta.client.describe_availability_zones()["AvailabilityZones"]:
        for subnet in vpc.subnets.filter(Filters=[{"Name": "availabilityZone", "Values": [az["ZoneName"]]}]):
            print(vpc, az["ZoneName"], "filter:", subnet)

(ec2.Vpc(id='vpc-88f051ef'), 'all:', ec2.Subnet(id='subnet-0b8ec121'))
(ec2.Vpc(id='vpc-88f051ef'), 'all:', ec2.Subnet(id='subnet-c9784df4'))
(ec2.Vpc(id='vpc-88f051ef'), 'all:', ec2.Subnet(id='subnet-4ef8b016'))
(ec2.Vpc(id='vpc-88f051ef'), 'all:', ec2.Subnet(id='subnet-4aaf903c'))
(ec2.Vpc(id='vpc-88f051ef'), 'all:', ec2.Subnet(id='subnet-28dc6424'))
(ec2.Vpc(id='vpc-88f051ef'), 'all:', ec2.Subnet(id='subnet-86bb27e3'))
(ec2.Vpc(id='vpc-88f051ef'), 'us-east-1a', 'filter:', ec2.Subnet(id='subnet-86bb27e3'))
(ec2.Vpc(id='vpc-88f051ef'), 'us-east-1b', 'filter:', ec2.Subnet(id='subnet-0b8ec121'))
(ec2.Vpc(id='vpc-88f051ef'), 'us-east-1c', 'filter:', ec2.Subnet(id='subnet-4aaf903c'))
(ec2.Vpc(id='vpc-88f051ef'), 'us-east-1d', 'filter:', ec2.Subnet(id='subnet-4ef8b016'))
(ec2.Vpc(id='vpc-88f051ef'), 'us-east-1e', 'filter:', ec2.Subnet(id='subnet-c9784df4'))
(ec2.Vpc(id='vpc-88f051ef'), 'us-east-1f', 'filter:', ec2.Subnet(id='subnet-28dc6424'))
(ec2.Vpc(id='vpc-f4250793'), 'all:', ec2.Subne

In [ ]:
ClientError: An error occurred (InvalidParameterValue) when calling the RunInstances operation: Value (eu-west-2a) for parameter availabilityZone is invalid. Subnet 'subnet-296efd64' is in the availability zone eu-west-2b
